# Interpolation Using images in video

In this notebook we will attempt to analyze an image and recreate it by using newton interpolation and Cubic Splines

# Funciones Basicas

In [8]:
import math as m
import numpy as np
from sympy import *

def f(f):
    x = Symbol("x")
    e = m.e
    pi = m.pi
    f = eval(f)
    return f
def g(g):
    x = Symbol("x")
    e = m.e
    pi = m.pi
    g = eval(g)
    return g
def f1(f):
    x = Symbol("x")
    e = m.e
    pi = m.pi
    f1 = diff(f)
    return f1
def f2(f1):
    x = Symbol("x")
    e = m.e
    pi = m.pi
    f2 = diff(f1)
    return f2
def evalfunct(f,y):
    x = Symbol("x")
    e = m.e
    pi = m.pi
    evalf = f.subs(x,y)
    return evalf
def regresiva(a,b):
    n = len(b)
    x = [0 for i in range(n)]
    x[n-1] = b[n-1,0]/a[n-1,n-1]
    for i in reversed(range(0,n-1)):
        suma = 0
        for j in range((i+1),n):
            suma = suma + a[i,j]*x[j]
        x[i] = (b[i,0] - suma)/a[i,i]
    return x
def progresiva(a,b):
    n = len(b)
    x = [0 for i in range(n)]
    x[0] = b[0,0]/a[0,0]
    for i in range(1,n):
        suma = 0
        for j in range(0,(i-1)):
            suma = suma + a[i,j]*x[j]
        x[i] = (b[i,0] - suma)/a[i,i]
    return x
def intercambioFila(a,fi1,fi2):
    f = np.copy(a)
    f[fi1,:] = a[fi2,:]
    f[fi2,:] = a[fi1,:]
    return f
def intercambioColumna(a,c1,c2):
    f = np.copy(a)
    f[:,[c1]] = a[:,[c2]]
    f[:,[c2]] = a[:,[c1]]
    return f

def pivoteoParcial(a,b,k):
    s = np.hstack(((a,b)))
    n = len(s) - 1
    mayor = abs(k)
    filaMayor = k
    for t in range(k,n):
        if abs(s[t,k]) > mayor:
            mayor = abs(s[t,k])
            filaMayor = t
    if mayor == 0:
        print("The system doesnt have a unique solution")
    elif filaMayor != k :
        s = intercambioFila(s,filaMayor,k)
    s = s[:,:-1]
    return s
def pivoteoTotal(a,b,k):
    s = np.hstack(((a,b)))
    n = len(a) - 1
    marcas = np.matrix('1 2 3 4')
    mayor = 0
    filaMayor = k
    columnaMayor = k
    for r in range (k,n):
        for t in range (k,n):
            if(abs(s[r,t]) > mayor):
                mayor = abs(s[r,t])
                filaMayor = r
                columnaMayor = t
    if mayor == 0:
        print("The system doesnt have a unique solution")
    elif filaMayor != k:
        s = intercambioFila(s,filaMayor,k)
    elif columnaMayor != k:
        s = intercambioColumna(s,columnaMayor,k)
        marcas = intercambioColumna(marcas,columnaMayor,k)
    s = s[:,:-1]
    print("The system is: ", marcas)
    return s
import numpy as np
#import matplotlib.pyplot as plt

def coef(x, y):
    x#.astype(float)
    y#.astype(float)
    n = len(x)
    a = []
    for i in range(n):
        a.append(y[i])

    for j in range(1, n):

        for i in range(n-1, j-1, -1):
            a[i] = (a[i]-a[i-1])/(x[i]-x[i-j])

    return np.array(a) # return an array of coefficient

def Eval(a, x, r):
    x#.astype(float)
    n = len( a ) - 1
    temp = a[n] + (r - x[n])
    for i in range( n - 1, -1, -1 ):
        temp = temp * ( r - x[i] ) + a[i]
    return temp # return the y_value interpolation

## Format Data

In order to process the data, we will format the pixels of each image as a numeric array

In [9]:
from PIL import Image
import pandas as pd
import numpy  as np
from scipy.interpolate import CubicSpline
import matplotlib.pyplot as plt

def get_colors(image):
    img = Image.open(image,'r')
    pix_val_r = list(img.getdata(0))
    pix_val_g = list(img.getdata(1))
    pix_val_b = list(img.getdata(2))
    df = pd.DataFrame(pix_val_r, columns =['r'])
    df.insert(1, value = pix_val_g, column = 'g')
    df.insert(2, value = pix_val_b, column = 'b')
    return df

### Getting images

We are currently using the dataset provided by The 2017 DAVIS Challenge (1). In this case, we will take two images from a video in order to interpolate them and compare the behaviour of the provided pixels. First, we get the images and formt them using the following functions. The result is a dataframe with RGB parameters for each pixel

In [10]:
img1 = 'DAVIS-2017-trainval-480p\\DAVIS\\JPEGImages\\480p\\bear\\00000.jpg'
img2 = 'DAVIS-2017-trainval-480p\\DAVIS\\JPEGImages\\480p\\bear\\00001.jpg'
img3 = 'DAVIS-2017-trainval-480p\\DAVIS\\JPEGImages\\480p\\bear\\00002.jpg'
img4 = 'DAVIS-2017-trainval-480p\\DAVIS\\JPEGImages\\480p\\bear\\00003.jpg'

df_img1 = get_colors(img1)
df_img2 = get_colors(img2)
df_img3 = get_colors(img3)
df_img4 = get_colors(img4)

#### Obtaining RGB dataframes

We will be suing this method with each of the color parameters of the pixels in the image to test

In [14]:
list_red_1 = df_img1['r']
list_red_2 = df_img2['r']
list_red_3 = df_img3['r']
list_red_4 = df_img4['r']

list_green_1 = df_img1['g']
list_green_2 = df_img2['g']
list_green_3 = df_img3['g']
list_green_4 = df_img4['g']

list_blue_1 = df_img1['b']
list_blue_2 = df_img2['b']
list_blue_3 = df_img3['b']
list_blue_4 = df_img4['b']

frames = '1 2 3 4'

## Data interpolation

using the dataset we selected three images from a video and got the numeric values for the colors of each pixel. In this case we will implemente multiple interpolation methods between image1 and image3

### DiffNewton

In [11]:
import numpy as np
#from funciones import *
#strx = str(raw_input("Please enter the vector of x: "))
#strfx = str(raw_input("Please enter the vector of Fx: "))
#strval = str(raw_input("Please enter the value to interpolate"))
def NewtonDiff(strx, strfx, strval):
#strx = '2 2.2 2.4 2.6 2.8'
    x = np.fromstring(strx, dtype=float, sep=' ')
#strfx = '-4.6109 -4.1749 -3.3768 -2.1362 -0.3553'
    fx = np.fromstring(strfx, dtype=float, sep=' ')
    #strval = '2.5'
    val = float(strval)
    n = len(x)
    a = []
    for i in range(n):
        a.append(fx[i])
    for j in range(1, n):
        for i in range(n-1, j-1, -1):
            a[i] = (a[i]-a[i-1])/(x[i]-x[i-j])
    a = np.array(a)
    n = len( a ) - 1
    temp = a[n] + (val - x[n])
    for i in range( n - 1, -1, -1 ):
        temp = temp * ( val - x[i] ) + a[i]
    result = temp
    return result

### B Cubic Splines

In [12]:
import numpy as np
from scipy import interpolate
from scipy.interpolate import CubicSpline
def BCubicSplines(strx, strfx, strval):
# Obtaining the data for the method
    x_points = np.fromstring(strx, dtype=float, sep=' ')
    y_points = np.fromstring(strfx, dtype=float, sep=' ')
    val = float(strval)

    #Method B Cubic Spline
    tck = interpolate.splrep(x_points, y_points)
    a = interpolate.splev(val, tck)
    return a

### Cubic Splines

In [13]:
import numpy as np
from scipy import interpolate
from scipy.interpolate import CubicSpline
def CubicSplines(strx, strfx, strval):
# Obtaining the data for the method
    x_points = np.fromstring(strx, dtype=float, sep=' ')
    y_points = np.fromstring(strfx, dtype=float, sep=' ')
    val = float(strval)
    
    #Method Cubic Spline
    cs = CubicSpline(x_points,y_points,bc_type='natural')
    return cs(val)

### Using Newton

In [15]:
i = 0
size = df_img1.shape[0] ## all frames have the same size
red_interpolation = []
green_interpolation = []
blue_interpolation = []

while(i < size):
    red_row = str(list_red_1[i]) + ' ' + str(list_red_2[i]) + ' ' + str(list_red_3[i]) + ' ' + str(list_red_4[i])
    red_interpolation.append(NewtonDiff(frames, red_row, 2.5))
    
    green_row = str(list_green_1[i]) + ' ' + str(list_green_2[i]) + ' ' + str(list_green_3[i]) + ' ' + str(list_green_4[i])
    green_interpolation.append(NewtonDiff(frames, green_row, 2.5))
    
    blue_row = str(list_blue_1[i]) + ' ' + str(list_blue_2[i]) + ' ' + str(list_blue_3[i]) + ' ' + str(list_blue_4[i])
    blue_interpolation.append(NewtonDiff(frames, blue_row, 2.5))
    i+= 1
df_interpolation = pd.DataFrame(red_interpolation, columns =['r'])
df_interpolation['g'] = green_interpolation
df_interpolation['b'] = blue_interpolation
df_interpolation.head()

,r,g,b
0,43.8125,38.8125,35.7500
1,41.6875,36.6875,33.6250
2,39.1875,34.1250,30.9375
3,46.8125,41.5625,38.3750
4,49.0000,44.8125,41.5625


### Cubic  B-Splines

In [16]:
i = 0
size = df_img1.shape[0] ## all frames have the same size
red_interpolation = []
green_interpolation = []
blue_interpolation = []

while(i < size):
    red_row = str(list_red_1[i]) + ' ' + str(list_red_2[i]) + ' ' + str(list_red_3[i]) + ' ' + str(list_red_4[i])
    red_interpolation.append(BCubicSplines(frames, red_row, 2.5))
    
    green_row = str(list_green_1[i]) + ' ' + str(list_green_2[i]) + ' ' + str(list_green_3[i]) + ' ' + str(list_green_4[i])
    green_interpolation.append(BCubicSplines(frames, green_row, 2.5))
    
    blue_row = str(list_blue_1[i]) + ' ' + str(list_blue_2[i]) + ' ' + str(list_blue_3[i]) + ' ' + str(list_blue_4[i])
    blue_interpolation.append(BCubicSplines(frames, blue_row, 2.5))
    i+= 1
df_interpolation = pd.DataFrame(red_interpolation, columns =['r'])
df_interpolation['g'] = green_interpolation
df_interpolation['b'] = blue_interpolation
df_interpolation.head()

,r,g,b
0,43.2500,38.25000000000001,35.1875
1,41.1250,36.125,33.0625
2,38.6250,33.5625,30.375
3,46.2500,41.0,37.8125
4,48.4375,44.25,41.00000000000001


### Cubic Splines

In [17]:
i = 0
size = df_img1.shape[0] ## all frames have the same size
red_interpolation = []
green_interpolation = []
blue_interpolation = []

while(i < size):
    red_row = str(list_red_1[i]) + ' ' + str(list_red_2[i]) + ' ' + str(list_red_3[i]) + ' ' + str(list_red_4[i])
    red_interpolation.append(CubicSplines(frames, red_row, 2.5))
    
    green_row = str(list_green_1[i]) + ' ' + str(list_green_2[i]) + ' ' + str(list_green_3[i]) + ' ' + str(list_green_4[i])
    green_interpolation.append(CubicSplines(frames, green_row, 2.5))
    
    blue_row = str(list_blue_1[i]) + ' ' + str(list_blue_2[i]) + ' ' + str(list_blue_3[i]) + ' ' + str(list_blue_4[i])
    blue_interpolation.append(CubicSplines(frames, blue_row, 2.5))
    i+= 1
df_interpolation = pd.DataFrame(red_interpolation, columns =['r'])
df_interpolation['g'] = green_interpolation
df_interpolation['b'] = blue_interpolation
df_interpolation.head()

,r,g,b
0,43.100,38.199999999999996,35.225
1,40.750,35.85,32.875
2,38.150,33.175000000000004,30.049999999999997
3,46.000,40.8,37.675000000000004
4,48.225,44.1,40.9


#  Printing the Interpolated Frame
Now that we have the result in a dataframe we can convert it into a matrix and then from the matrix format we can turn it into a jpg using PIL

In [ ]:
# Size in pixels of the image (resolution)
size_row = 480
size_col = 854
interpolated_matrix = np.ndarray((size_row, size_col, 3),dtype=np.uint8)
i = 0
j = 0
pos = 0
while i < size_row:
    while j < size_col:
        interpolated_matrix[i][j][0] = df_interpolation['r'][pos]
        interpolated_matrix[i][j][1] = df_interpolation['g'][pos]
        interpolated_matrix[i][j][2] = df_interpolation['b'][pos]
        pos += 1
        j += 1
    i += 1
    j = 0
#interpolated_matrix
#img_result = Image.fromarray(interpolated_matrix, 'RGB')
#img_result.show()

In [ ]:
#test_matrix
img_result = Image.fromarray(interpolated_matrix, 'RGB')
#img_result.save('name.jpg') #the name of the file to save
#img_result.show()